# Postresql Links

https://popsql.com/learn-sql/postgresql/how-to-query-a-json-column-in-postgresql

In [1]:
from sqlalchemy import create_engine
from sqlalchemy import MetaData, cast, select
from sqlalchemy.engine import Engine

from mangostar import module_settings as settings
from sql_queries.tables import MetaTableAdapter
from sql_queries.materialized import CreateView
# from sql_queries
from sql_queries.utils import execute
# from sql_queries.materialized import CreateView
from sqlalchemyplus import MaterializedView, CreateMaterializedView
from sqlalchemy.schema import CreateTable
from sql_queries.timescale import time_bucket
from pydantic import validate_arguments
from datetime import datetime
from faker_schema.faker_schema import FakerSchema
from faker import Faker
from sqlalchemy.dialects import postgresql
from sqlalchemy.dialects.postgresql import UUID
from sqlalchemy import Index
from contextlib import suppress


In [1]:
from mangostar import module_settings as settings

In [2]:
engine = create_engine(settings.postgres_connection_str)
metadata = MetaData(bind = engine)
adapter = MetaTableAdapter(metadata)
faker_schema = FakerSchema()
faker = Faker()

In [3]:
# print(CreateTable(adapter.kernel))

# Adding an employee record and intend to run some experiments on it to test for selection statements.

In [4]:
flat_schema = {
    'employee_id': 'uuid4',
    'employee_name': 'name',
    'employee address': 'address',
    'email_address': 'email'
}

In [5]:
kernel = adapter.kernel

In [6]:
metadata.drop_all()


In [7]:
metadata.create_all()
idx = Index("idx_kernel", kernel.c.created_at, kernel.c.data, kernel.c.event_at, kernel.c.tags)
with suppress(Exception):
    idx.create(engine)

In [8]:
def gen_employee():
    return faker_schema.generate_fake(flat_schema)

In [9]:
@validate_arguments
def insert_to(*,bucket:str, tags: dict={}, event_at: datetime=datetime.now(), **values):
    kernel.insert().values(bucket=bucket, tags=tags, event_at=event_at, **values).execute()


In [10]:
def generate_and_insert_employee():
    insert_to(bucket="acme_employee", tags={"tenant_name": "acme corp"}, event_at=faker.date_time_this_month(), data=gen_employee())

In [11]:

# kernel = adapter.kernel.create(postgresql)

In [12]:
def gen_employee():
    return faker_schema.generate_fake(flat_schema)

In [13]:
gen_employee()

{'employee_id': '7af1fa77-5ac1-49f6-b81d-6e7911d4379a',
 'employee_name': 'Sean Rodriguez',
 'employee address': '78877 Tammy Camp Apt. 362\nMcfarlandborough, OR 75877',
 'email_address': 'braunjoshua@yahoo.com'}

In [14]:
# Probably will wrap functions in something like this to give orm like experience.
@validate_arguments
def insert_to(*,bucket:str, tags: dict={}, event_at: datetime=datetime.now(), **values):
    kernel.insert_into(is_execute=True, bucket=bucket, tags=tags, event_at=event_at)


In [15]:
def generate_and_insert_employee():
    insert_to(bucket="acme_employee", tags={"tenant_name": "acme corp"}, event_at=faker.date_time_this_month(), data=gen_employee())

In [18]:
generate_and_insert_employee()

In [ ]:
data_table.c.data[('key_1', 'key_2', 5, ..., 'key_n')]



In [19]:
eid_column = cast(kernel.data[('key_1', 'key_2', 5, ..., 'key_n')].astext, UUID).label("employee_id")
select_stmt = select([eid_column]).where(kernel.c.bucket=="acme_employee")
CreateView("employee_table", select_stmt).execute(bind=engine)

KeyError: 'key_1, key_2, 5, Ellipsis, key_n'

In [29]:
select_stmt = select([eid_column]).where(kernel.c.bucket=="acme_employee").compile(bind=engine, compile_kwargs = {"literal_binds": True})
select_stmt_str:str = str(select_stmt)
select_stmt_str.replace("\n", "")

"SELECT CAST(kernel.data -> 'employee_id' AS UUID) AS employee_id FROM kernel WHERE kernel.bucket = 'acme_employee'"

In [13]:
print()

In [14]:
select_stmt_str.replace("\n", "")

"SELECT CAST(kernel.data -> 'employee_id' AS UUID) AS employee_id, NULL AS anon_1 FROM kernel WHERE kernel.bucket = 'acme_employee'"

In [20]:
def create_nested_path(nested_list: list, element: str):
    composite = "$"
    for i in nested_list:
        composite += f".{i}[*]"
    return f"{composite}.{element}"

In [21]:
create_nested_path(["one", "two", "three"], "four")

'$.one[*].two[*].three[*].four'